In [13]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
import cv2
import numpy as np

# ==============================
# ✅ SETTINGS
# ==============================
dataset_dir = r"C:\Users\shivain\Downloads\Face Mask Dataset"  # single folder with WithMask & WithoutMask
img_size = 96
batch_size = 8
epochs = 3
model_path = "mask_detector_model.h5"

# ==============================
# ✅ STEP 1: VERIFY DATASET PATH
# ==============================
if not os.path.exists(dataset_dir):
    raise FileNotFoundError(f"❌ Dataset not found at: {dataset_dir}")
else:
    print("✅ Dataset found at:", dataset_dir)
    print("📂 Subfolders:", os.listdir(dataset_dir))

# ==============================
# ✅ STEP 2: AUTO TRAIN/VAL SPLIT
# ==============================
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 80% training, 20% validation
)

train_gen = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# ==============================
# ✅ STEP 3: BUILD MODEL (TRANSFER LEARNING)
# ==============================
base_model = MobileNetV2(input_shape=(img_size, img_size, 3),
                         include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze pre-trained layers

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ==============================
# ✅ STEP 4: TRAIN MODEL
# ==============================
print("\n✅ Starting training...")
model.fit(train_gen, validation_data=val_gen, epochs=epochs)

# Save the trained model
model.save(model_path)
print(f"✅ Training complete! Model saved as {model_path}")

# ==============================
# ✅ STEP 5: REAL-TIME MASK DETECTION
# ==============================
print("\n🎥 Starting real-time mask detection...")
model = tf.keras.models.load_model(model_path)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess frame for model
    img = cv2.resize(frame, (img_size, img_size))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    # Predict mask/no-mask
    prediction = model.predict(img, verbose=0)[0][0]
    label = "Mask" if prediction < 0.5 else "No Mask"
    color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

    # Show result on webcam feed
    cv2.putText(frame, label, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    cv2.rectangle(frame, (10, 10), (frame.shape[1]-10, frame.shape[0]-10), color, 2)

    cv2.imshow("Face Mask Detection", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("✅ Webcam closed. Program finished.")


✅ Dataset found at: C:\Users\shivain\Downloads\Face Mask Dataset
📂 Subfolders: ['mask_detector_model.h5', 'realtime_mask_detector.h5', 'realtime_mask_detector.py', 'Test', 'Train', 'train_face_mask.ipynb', 'Validation']
Found 1187 images belonging to 3 classes.
Found 295 images belonging to 3 classes.
9406464/9406464 [==============================] - 3s 0us/step

✅ Starting training...
Epoch 1/3
149/149 [==============================] - 17s 91ms/step - loss: 0.3947 - accuracy: 0.4153 - val_loss: 0.3447 - val_accuracy: 0.4136
Epoch 2/3
149/149 [==============================] - 8s 54ms/step - loss: 0.2254 - accuracy: 0.4254 - val_loss: 0.2454 - val_accuracy: 0.4237
Epoch 3/3
149/149 [==============================] - 8s 55ms/step - loss: 0.1115 - accuracy: 0.4263 - val_loss: 0.2981 - val_accuracy: 0.4407
✅ Training complete! Model saved as mask_detector_model.h5

🎥 Starting real-time mask detection...
✅ Webcam closed. Program finished.
